# Getting Started

This notebook walks through how LangChain thinks about memory. 

Memory involves keeping a concept of state around throughout a user's interactions with an language model. A user's interactions with a language model are captured in the concept of ChatMessages, so this boils down to ingesting, capturing, transforming and extracting knowledge from a sequence of chat messages. There are many different ways to do this, each of which exists as its own memory type.

In general, for each type of memory there are two ways to understanding using memory. These are the standalone functions which extract information from a sequence of messages, and then there is the way you can use this type of memory in a chain. 

Memory can return multiple pieces of information (for example, the most recent N messages and a summary of all previous messages). The returned information can either be a string or a list of messages.

In this notebook, we will walk through the simplest form of memory: "buffer" memory, which just involves keeping a buffer of all prior messages. We will show how to use the modular utility functions here, then show how it can be used in a chain (both returning a string as well as a list of messages).

## ChatMessageHistory
One of the core utility classes underpinning most (if not all) memory modules is the `ChatMessageHistory` class. This is a super lightweight wrapper which exposes convienence methods for saving Human messages, AI messages, and then fetching them all. 

You may want to use this class directly if you are managing memory outside of a chain.

In [2]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [3]:
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False)]

## ConversationBufferMemory

We now show how to use this simple concept in a chain. We first showcase `ConversationBufferMemory` which is just a wrapper around ChatMessageHistory that extracts the messages in a variable.

We can first extract it as a string.

In [4]:
from langchain.memory import ConversationBufferMemory

In [5]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [6]:
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: whats up?'}

We can also get the history as a list of messages

In [7]:
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!', additional_kwargs={}, example=False),
  AIMessage(content='whats up?', additional_kwargs={}, example=False)]}

## Saving Message History

You may often have to save messages, and then load them to use again. This can be done easily by first converting the messages to normal python dictionaries, saving those (as json or something) and then loading those. Here is an example of doing that.

In [9]:
import json

from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [10]:
dicts = messages_to_dict(history.messages)

In [11]:
dicts

[{'type': 'human',
  'data': {'content': 'hi!', 'additional_kwargs': {}, 'example': False}},
 {'type': 'ai',
  'data': {'content': 'whats up?', 'additional_kwargs': {}, 'example': False}}]

In [12]:
new_messages = messages_from_dict(dicts)

In [13]:
new_messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False)]

## Using Memory

You can use a memory on a `Chain` object that accepts a memory object such as `LLMChain`. This is the recommended way in most use cases.

In [14]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = """\
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:
"""

memory = ConversationBufferMemory()
llm = OpenAI(temperature=0)
conversation = LLMChain(
    prompt=PromptTemplate.from_template(prompt),
    llm=llm, 
    verbose=True, 
    memory=memory
)

In [15]:
conversation.predict(input="Hi there!")



> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:


> Finished chain.


"Hello! It's nice to meet you. How can I help you today?"

In [16]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:


> Finished chain.


"That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

Under the hood, `LLMChain` format the prompt template using both the input variables passed by the users as well as the memory key values provided by the memory object. You can inspect the memory variable name through `memory_variables` property.

In [17]:
memory.memory_variables

['history']

If you wish to have a different memory key name, it is recommended to do it during initialization. 

In [18]:
prompt = """\
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{conversation_history}
Human: {input}
AI:
"""

memory = ConversationBufferMemory(memory_key="conversation_history")
llm = OpenAI(temperature=0)
conversation = LLMChain(
    prompt=PromptTemplate.from_template(prompt),
    llm=llm, 
    verbose=True, 
    memory=memory
)

In [20]:
conversation.predict(input="Hello")



> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello
AI:


> Finished chain.


"Hello there! It's nice to meet you. How can I help you today?"

Aside from using it in a `Chain`, you can also use a memory by directly calling its exposed methods (`load_memory_variables` and `save_context`). It is not recommended to do so unless you have good understanding of the underlying implementation of the memory class. 

In [21]:
memory.load_memory_variables({})

{'conversation_history': "Human: Hello\nAI: Hello there! It's nice to meet you. How can I help you today?"}

## Category of memory

Conceptually, it is helpful to categorize the memory available in LangChain into three categories, based on their target use cases.

1. Chat Message Memory

Majority of the memory class in LangChain is chat message memory. This class of memory is specialized in storing chat message history in a conversation with an LLM model. The class wraps around a `BaseChatMessageHistory` object, controlling how the chat message history is presented to the LLM model.

You can easily distinguish this type of memory class through the presence of `chat_memory` field during initialization. Essentially, `chat_memory` variable holds the `BaseChatMessageHistory` object being wrapped by the memory class.

In [22]:
from langchain.memory import ConversationBufferMemory, ChatMessageHistory

chat_history = ChatMessageHistory()

# ConversationBufferMemory is a chat message memory object.
memory = ConversationBufferMemory(
    chat_memory=chat_history # <-- This is the `BaseChatMessageHistory` object
)

In [25]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

conversation = ConversationChain(
    llm=ChatOpenAI(temperature=0),
    memory=memory,
    verbose=True
)

In [26]:
conversation.run("Hello!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


'Hello there! How can I assist you today?'

In [27]:
conversation.run("How is your day?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello!
AI: Hello there! How can I assist you today?
Human: How is your day?
AI:

> Finished chain.


'As an AI, I don\'t really have "days" in the traditional sense, but I\'m functioning well and ready to help you with whatever you need. How can I assist you today?'

There is a lot more about chat message memory. You can learn more [here](../tutorials/chat_message_history.ipynb).

2. Generic Memory

Memory class that is not tied to any use case. You can use this type of memory class to store just about anything.

You can find a list of generic memory classes [here](./how_to_guides.rst).

3. Decorator Memory

Decorator memory wraps around other memory classes and add additional functionalities to them, much like a Python decorator.

You can find a list of decorator memory classes [here](./how_to_guides.rst).